In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier


wl 할때 2008년상관안쓰고 한것

In [3]:
# 데이터 불러오기
df = pd.read_csv("/content/american_bankruptcy_dataset.csv")


#열 이름 변경
col_mapping = {
    'X1': '유동자산', 'X2': '매출원가', 'X3': '감가상각비', 'X4': 'EBITDA',
    'X5': '재고자산', 'X6': '당기순이익', 'X7': '매출채권', 'X8': '시가총액',
    'X9': '순매출', 'X10': '총자산', 'X11': '장기부채', 'X12': 'EBIT',
    'X13': '매출총이익', 'X14': '유동부채', 'X15': '이익잉여금',
    'X16': '총수익', 'X17': '총부채', 'X18': '영업비용총합',
}
df = df.rename(columns=col_mapping)


# 라벨 처리 및 정렬
df["파산여부"] = df["status_label"].map({"alive": 0, "failed": 1})
df = df.sort_values(by=["company_name", "fyear"]).reset_index(drop=True)


from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

def get_basic_split(df, wl=5, seed=42):
    features = [
        '유동자산', '매출원가', '감가상각비', 'EBITDA', '재고자산', '당기순이익', '매출채권',
        '시가총액', '순매출', '총자산', '장기부채', 'EBIT', '매출총이익', '유동부채',
        '이익잉여금', '총수익', '총부채', '영업비용총합'
    ]

    X_list, y_list, split_list = [], [], []

    for _, group in df.groupby("company_name"):
        group = group.sort_values("fyear")
        X = group[features].values
        y = group["파산여부"].values
        years = group["fyear"].values

        for i in range(len(group) - wl):
            x_window = X[i:i+wl].flatten()
            y_target = y[i+wl]
            year_target = years[i+wl]

            if year_target <= 2011:
                split = "train"
            elif 2012 <= year_target <= 2014:
                split = "valid"
            elif 2015 <= year_target <= 2018:
                split = "test"
            else:
                continue

            X_list.append(x_window)
            y_list.append(y_target)
            split_list.append(split)

    df_task = pd.DataFrame({
        "X": X_list,
        "label": y_list,
        "split": split_list
    })

    train_df = df_task[df_task["split"] == "train"]
    test_df  = df_task[df_task["split"] == "test"]

    # 언더샘플링
    bankrupt = train_df[train_df["label"] == 1]
    healthy = train_df[train_df["label"] == 0].sample(n=len(bankrupt), random_state=seed)
    train_balanced = pd.concat([bankrupt, healthy]).sample(frac=1, random_state=seed)

    # feature matrix
    X_train = np.stack(train_balanced["X"].values)
    y_train = train_balanced["label"].values
    X_test  = np.stack(test_df["X"].values)
    y_test  = test_df["label"].values

    # 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    return X_train_scaled, y_train, X_test_scaled, y_test


wl 할 떄 금융위기였던 2008년 특정짓기

In [ ]:
# 전체 통합 전처리 흐름 + 수정된 함수까지 포함

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. 데이터 불러오기 및 열 이름 변경
df = pd.read_csv("/content/american_bankruptcy_dataset.csv")

col_mapping = {
    'X1': '유동자산', 'X2': '매출원가', 'X3': '감가상각비', 'X4': 'EBITDA',
    'X5': '재고자산', 'X6': '당기순이익', 'X7': '매출채권', 'X8': '시가총액',
    'X9': '순매출', 'X10': '총자산', 'X11': '장기부채', 'X12': 'EBIT',
    'X13': '매출총이익', 'X14': '유동부채', 'X15': '이익잉여금',
    'X16': '총수익', 'X17': '총부채', 'X18': '영업비용총합',
}
df = df.rename(columns=col_mapping)
df["파산여부"] = df["status_label"].map({"alive": 0, "failed": 1})
df = df.sort_values(by=["company_name", "fyear"]).reset_index(drop=True)

# 2. WL 기반 데이터셋 생성 함수 (is_crisis 포함)
def prepare_task1_dataset(df, wl=5):
    features = [
        '유동자산', '매출원가', '감가상각비', 'EBITDA', '재고자산', '당기순이익', '매출채권',
        '시가총액', '순매출', '총자산', '장기부채', 'EBIT', '매출총이익', '유동부채',
        '이익잉여금', '총수익', '총부채', '영업비용총합'
    ]

    X_list, y_list, split_list, crisis_flag_list = [], [], [], []

    for _, group in df.groupby("company_name"):
        group = group.sort_values("fyear")
        X = group[features].values
        y = group["파산여부"].values
        years = group["fyear"].values

        for i in range(len(group) - wl):
            x_window = X[i:i+wl].flatten()
            y_target = y[i+wl]
            year_target = years[i+wl]

            if year_target <= 2011:
                split = "train"
            elif 2012 <= year_target <= 2014:
                split = "valid"
            elif 2015 <= year_target <= 2018:
                split = "test"
            else:
                continue

            X_list.append(x_window)
            y_list.append(y_target)
            split_list.append(split)
            crisis_flag_list.append(1 if year_target == 2008 else 0)

    df_result = pd.DataFrame({
        "features": X_list,
        "label": y_list,
        "split": split_list,
        "is_crisis": crisis_flag_list
    })

    return df_result

# 3. 반복 실험 대응용 전처리 함수
def get_balanced_split_with_crisis(df, wl=5, seed=42):
    df_task = prepare_task1_dataset(df, wl=wl)
    df_task["X"] = df_task.apply(lambda row: np.append(row["features"], row["is_crisis"]), axis=1)

    train_df = df_task[df_task["split"] == "train"]
    valid_df = df_task[df_task["split"] == "valid"]
    test_df  = df_task[df_task["split"] == "test"]

    bankrupt = train_df[train_df["label"] == 1]
    healthy = train_df[train_df["label"] == 0].sample(n=len(bankrupt), random_state=seed)
    train_balanced = pd.concat([bankrupt, healthy]).sample(frac=1, random_state=seed).reset_index(drop=True)

    X_train = np.stack(train_balanced["X"].values)
    y_train = train_balanced["label"].values
    X_valid = np.stack(valid_df["X"].values)
    y_valid = valid_df["label"].values
    X_test  = np.stack(test_df["X"].values)
    y_test  = test_df["label"].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    X_test_scaled  = scaler.transform(X_test)

    return X_train_scaled, y_train, X_valid_scaled, y_valid, X_test_scaled, y_test


wl을 여러가지 비교해서 하는 코드(wl 어떤 값으로 하면 좋을지)

In [ ]:
# Now that the file has been uploaded, we retry the WL=1~5 preprocessing and evaluation

# Load the uploaded dataset
df = pd.read_csv("/content/american_bankruptcy_dataset.csv")

# Rename columns
col_mapping = {
    'X1': '유동자산', 'X2': '매출원가', 'X3': '감가상각비', 'X4': 'EBITDA',
    'X5': '재고자산', 'X6': '당기순이익', 'X7': '매출채권', 'X8': '시가총액',
    'X9': '순매출', 'X10': '총자산', 'X11': '장기부채', 'X12': 'EBIT',
    'X13': '매출총이익', 'X14': '유동부채', 'X15': '이익잉여금',
    'X16': '총수익', 'X17': '총부채', 'X18': '영업비용총합',
}
df = df.rename(columns=col_mapping)

# Label mapping and sorting
df["파산여부"] = df["status_label"].map({"alive": 0, "failed": 1})
df = df.sort_values(by=["company_name", "fyear"]).reset_index(drop=True)

# Define function to create WL dataset
def prepare_task1_dataset(df, wl=1):
    features = [
        '유동자산', '매출원가', '감가상각비', 'EBITDA', '재고자산', '당기순이익', '매출채권',
        '시가총액', '순매출', '총자산', '장기부채', 'EBIT', '매출총이익', '유동부채',
        '이익잉여금', '총수익', '총부채', '영업비용총합'
    ]
    X_list, y_list, split_list = [], [], []

    for _, group in df.groupby("company_name"):
        group = group.sort_values("fyear")
        X = group[features].values
        y = group["파산여부"].values
        years = group["fyear"].values

        for i in range(len(group) - wl):
            x_window = X[i:i+wl].flatten()
            y_target = y[i+wl]
            year_target = years[i+wl]

            if year_target <= 2011:
                split = "train"
            elif 2012 <= year_target <= 2014:
                split = "valid"
            elif 2015 <= year_target <= 2018:
                split = "test"
            else:
                continue

            X_list.append(x_window)
            y_list.append(y_target)
            split_list.append(split)

    return pd.DataFrame({
        "features": X_list,
        "label": y_list,
        "split": split_list
    })

# Undersample function
def undersample(df):
    bankrupt = df[df["label"] == 1]
    healthy = df[df["label"] == 0].sample(n=len(bankrupt), random_state=42)
    return pd.concat([bankrupt, healthy]).sample(frac=1, random_state=42).reset_index(drop=True)

# Model evaluation function
def evaluate_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return {
        "AUC": auc, "Recall": recall, "Precision": precision, "F1": f1,
        "TP": cm[1, 1], "FP": cm[0, 1], "FN": cm[1, 0], "TN": cm[0, 0]
    }

# Run for WL=1 to 6
results = []
for wl in range(1, 6):  # 1~5까지만 실행
    df_task = prepare_task1_dataset(df, wl=wl)
    df_task["X"] = df_task["features"].apply(lambda x: np.array(x))

    train_df = df_task[df_task["split"] == "train"]
    test_df = df_task[df_task["split"] == "test"]
    train_balanced = undersample(train_df)

    X_train = np.stack(train_balanced["X"].values)
    y_train = train_balanced["label"].values
    X_test = np.stack(test_df["X"].values)
    y_test = test_df["label"].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    metrics = evaluate_model(X_train_scaled, y_train, X_test_scaled, y_test)
    metrics["WL"] = wl
    results.append(metrics)

df_results = pd.DataFrame(results).sort_values("WL")
df_results  # 최종 출력



,AUC,Recall,Precision,F1,TP,FP,FN,TN,WL
0,0.738752,0.702128,0.045559,0.085566,198,4148,84,7384,1
1,0.755895,0.724265,0.050865,0.095054,197,3676,75,7249,2
2,0.752702,0.701550,0.048578,0.090863,181,3545,77,6714,3
3,0.764306,0.730290,0.052381,0.097751,176,3184,65,6426,4
4,0.774351,0.734234,0.052010,0.097139,163,2971,59,6080,5


In [ ]:
#  Ranodom Forest 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model =RandomForestClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary_basic = {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}
summary_basic

{'AUC': '0.7643',
 'F1': '0.0947',
 'Recall': '0.7237',
 'Precision': '0.0507',
 'Type1': '0.3326',
 'Type2': '0.2763'}

In [ ]:
# Random Forest 모델 30회 성능 평가(crisis포함)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

results = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, _, _, X_test, y_test = get_balanced_split_with_crisis(df, wl=5, seed=repeat)

    model= RandomForestClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results["AUC"].append(auc)
    results["F1"].append(f1)
    results["Recall"].append(recall)
    results["Precision"].append(precision)
    results["Type1"].append(type1)
    results["Type2"].append(type2)

# 평균 성능 출력
summary = {metric: f"{np.mean(values):.4f}" for metric, values in results.items()}
summary


{'AUC': '0.7658',
 'F1': '0.0953',
 'Recall': '0.7282',
 'Precision': '0.0510',
 'Type1': '0.3327',
 'Type2': '0.2718'}

In [ ]:
#  Logistic Regression 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model = LogisticRegression(max_iter=1000, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary_basic = {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}
summary_basic

{'AUC': '0.6718',
 'F1': '0.0590',
 'Recall': '0.7158',
 'Precision': '0.0307',
 'Type1': '0.5539',
 'Type2': '0.2842'}

In [ ]:
# logistic regression 모델 30회 성능 평가(crisis포함)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

results = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, _, _, X_test, y_test = get_balanced_split_with_crisis(df, wl=5, seed=repeat)

    model=LogisticRegression(max_iter=1000,random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results["AUC"].append(auc)
    results["F1"].append(f1)
    results["Recall"].append(recall)
    results["Precision"].append(precision)
    results["Type1"].append(type1)
    results["Type2"].append(type2)

# 평균 성능 출력
summary = {metric: f"{np.mean(values):.4f}" for metric, values in results.items()}
summary


{'AUC': '0.6717',
 'F1': '0.0582',
 'Recall': '0.7237',
 'Precision': '0.0303',
 'Type1': '0.5678',
 'Type2': '0.2763'}

In [ ]:
#  GradientBoosting 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model = GradientBoostingClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary_basic = {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}

NameError: name 'GradientBoostingClassifier' is not defined

In [ ]:
#  XGboost 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model = XGBClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary_basic = {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}

In [ ]:
# SVM 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model = SVC(probability=True, random_state=repeat)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary_basic = {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}

In [ ]:
#  ANN 30회 성능 평가
results_basic = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)
    model = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=1000,
    early_stopping=True,
    random_state=repeat
)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results_basic["AUC"].append(auc)
    results_basic["F1"].append(f1)
    results_basic["Recall"].append(recall)
    results_basic["Precision"].append(precision)
    results_basic["Type1"].append(type1)
    results_basic["Type2"].append(type2)

# 평균 성능 출력
summary= {metric: f"{np.mean(values):.4f}" for metric, values in results_basic.items()}
summary

{'AUC': '0.7231',
 'F1': '0.0835',
 'Recall': '0.7077',
 'Precision': '0.0444',
 'Type1': '0.3784',
 'Type2': '0.2923'}

In [ ]:
# ANN 30회 성능 평가(crisis포함)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

results = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, _, _, X_test, y_test = get_balanced_split_with_crisis(df, wl=5, seed=repeat)

    model = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=1000,
    early_stopping=True,
    random_state=repeat
)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results["AUC"].append(auc)
    results["F1"].append(f1)
    results["Recall"].append(recall)
    results["Precision"].append(precision)
    results["Type1"].append(type1)
    results["Type2"].append(type2)

# 평균 성능 출력
summary = {metric: f"{np.mean(values):.4f}" for metric, values in results.items()}
summary

{'AUC': '0.7269',
 'F1': '0.0814',
 'Recall': '0.7125',
 'Precision': '0.0432',
 'Type1': '0.3925',
 'Type2': '0.2875'}

심화 모델링 :  회귀 + 분류 혼합 모델

In [ ]:
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix

results = {"AUC": [], "F1": [], "Recall": [], "Precision": [], "Type1": [], "Type2": []}

for repeat in range(30):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # 1단계: 확률 회귀 예측
    reg = XGBRegressor(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=repeat)
    reg.fit(X_train, y_train)

    train_prob = reg.predict(X_train).reshape(-1, 1)
    test_prob = reg.predict(X_test).reshape(-1, 1)

    # 2단계: 분류기 학습
    clf = LogisticRegression(max_iter=1000, random_state=repeat)
    clf.fit(train_prob, y_train)

    y_pred = clf.predict(test_prob)
    y_prob = clf.predict_proba(test_prob)[:, 1]

    auc = roc_auc_score(y_test, y_prob)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    type1 = fp / (fp + tn)
    type2 = fn / (fn + tp)

    results["AUC"].append(auc)
    results["F1"].append(f1)
    results["Recall"].append(recall)
    results["Precision"].append(precision)
    results["Type1"].append(type1)
    results["Type2"].append(type2)

# 결과 요약
summary = {metric: f"{np.mean(values):.4f}" for metric, values in results.items()}
print(summary)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:24:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:25:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:25:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:25:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:25:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

{'AUC': '0.7301', 'F1': '0.0877', 'Recall': '0.6770', 'Precision': '0.0469', 'Type1': '0.3375', 'Type2': '0.3230'}


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd

# 1. 100회 반복 후 예측 확률 저장
all_probs = []
for repeat in range(100):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # 논문 기반 ANN 구조 예시: 2 hidden layers (64, 32), ReLU, max_iter=300
    model = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=1000,
    early_stopping=True,
    random_state=repeat
)
    model.fit(X_train, y_train)

    prob = model.predict_proba(X_test)[:, 1]
    all_probs.append(prob)

# 2. 평균 확률 계산
mean_prob = np.mean(all_probs, axis=0)

# 3. 등급 매핑
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

# 4. 등급별 실제 부도율 비교 (Observed Default Rate)
grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test
})

summary = grade_df.groupby("grade").agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

# 결과 출력
summary


<ipython-input-30-fd9847469025>:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade").agg(


,grade,n,default,expected_prob,odr,abs_diff,wmae
0,AAA,143,0,0.009831,0.000000,0.009831,0.000152
1,AA,164,0,0.033825,0.000000,0.033825,0.000598
2,A,220,0,0.074213,0.000000,0.074213,0.001761
3,BBB,693,7,0.177410,0.010101,0.167309,0.012503
4,B,8053,215,0.501759,0.026698,0.475060,0.412559


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd

# 1. 100회 반복 후 예측 확률 저장
all_probs = []
for repeat in range(100):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # 논문 기반 randomforest구조
    model=RandomForestClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    prob = model.predict_proba(X_test)[:, 1]
    all_probs.append(prob)

# 2. 평균 확률 계산
mean_prob = np.mean(all_probs, axis=0)

# 3. 등급 매핑
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

# 4. 등급별 실제 부도율 비교 (Observed Default Rate)
grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test
})

summary = grade_df.groupby("grade").agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

# 결과 출력
summary

<ipython-input-31-58b221c963e7>:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade").agg(


,grade,n,default,expected_prob,odr,abs_diff,wmae
0,AAA,6,0,0.017350,0.000000,0.017350,0.000011
1,AA,92,0,0.039240,0.000000,0.039240,0.000389
2,A,268,0,0.074986,0.000000,0.074986,0.002167
3,BBB,1119,2,0.189792,0.001787,0.188004,0.022687
4,B,7788,220,0.470971,0.028249,0.442722,0.371824


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd

# 1. 100회 반복 후 예측 확률 저장
all_probs = []
for repeat in range(100):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # 논문 기반 XGBoost 구조
    model = XGBClassifier(n_estimators=100, random_state=repeat)
    model.fit(X_train, y_train)

    prob = model.predict_proba(X_test)[:, 1]
    all_probs.append(prob)

# 2. 평균 확률 계산
mean_prob = np.mean(all_probs, axis=0)

# 3. 등급 매핑
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

# 4. 등급별 실제 부도율 비교 (Observed Default Rate)
grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test
})

summary = grade_df.groupby("grade").agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

# 결과 출력
summary


<ipython-input-33-dc531151639e>:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade").agg(


,grade,n,default,expected_prob,odr,abs_diff,wmae
0,AAA,430,0,0.010998,0.000000,0.010998,0.000510
1,AA,627,2,0.034383,0.003190,0.031194,0.002109
2,A,901,3,0.073994,0.003330,0.070664,0.006866
3,BBB,2037,16,0.170775,0.007855,0.162920,0.035789
4,B,5278,201,0.554668,0.038083,0.516586,0.294030


In [ ]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd

# 더미 데이터 생성 (실제 사용 시 df를 사용자 데이터로 교체)
np.random.seed(0)
n = 1000
df = pd.DataFrame({
    'feature1': np.random.randn(n),
    'feature2': np.random.randn(n),
    'feature3': np.random.randn(n),
    'target': np.random.binomial(1, 0.1, size=n)
})

def get_basic_split(df, wl=5, seed=0):
    np.random.seed(seed)
    shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    split = int(0.7 * len(shuffled))
    train = shuffled[:split]
    test = shuffled[split:]
    X_train = train.drop(columns="target")
    y_train = train["target"]
    X_test = test.drop(columns="target")
    y_test = test["target"]
    return X_train, y_train, X_test, y_test

# 혼합모델 (XGBClassifier → MLPClassifier) 기반 등급 매핑
all_probs = []
for repeat in range(100):  # 반복 횟수 줄임
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # 1단계: XGBoost 분류기
    base_model = XGBClassifier(n_estimators=10, use_label_encoder=False, eval_metric='logloss', random_state=repeat)
    base_model.fit(X_train, y_train)

    train_prob = base_model.predict_proba(X_train)[:, 1].reshape(-1, 1)
    test_prob = base_model.predict_proba(X_test)[:, 1].reshape(-1, 1)

    # 2단계: MLPClassifier로 혼합
    mix_model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=1000, random_state=repeat)
    mix_model.fit(train_prob, y_train)

    final_prob = mix_model.predict_proba(test_prob)[:, 1]
    all_probs.append(final_prob)

# 평균 확률 계산
mean_prob = np.mean(all_probs, axis=0)

# 등급 매핑
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

# 등급별 실제 부도율 비교
grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test.values  # 마지막 반복 기준
})

summary = grade_df.groupby("grade").agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

print(summary)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:07:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:07:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:07:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:07:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:07:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

  grade    n  default  expected_prob       odr  abs_diff      wmae
0   AAA    0        0            NaN       NaN       NaN       NaN
1    AA   65        5       0.042927  0.076923  0.033996  0.007366
2     A  234       18       0.064539  0.076923  0.012384  0.009659
3   BBB    1        0       0.106900  0.000000  0.106900  0.000356
4     B    0        0            NaN       NaN       NaN       NaN


<ipython-input-42-c75e98555032>:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade").agg(


보정된 ANN

In [ ]:
# 혼합 모델: ANN → Logistic Regression
all_probs = []

for repeat in range(100):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    # Stage 1: ANN으로 확률 예측
    ann = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu',
                        solver='adam', max_iter=1000, early_stopping=True, random_state=repeat)
    ann.fit(X_train, y_train)
    train_prob = ann.predict_proba(X_train)[:, 1].reshape(-1, 1)
    test_prob = ann.predict_proba(X_test)[:, 1].reshape(-1, 1)

    # Stage 2: Logistic Regression으로 보정
    lr = LogisticRegression(max_iter=1000, random_state=repeat)
    lr.fit(train_prob, y_train)
    final_prob = lr.predict_proba(test_prob)[:, 1]

    all_probs.append(final_prob)

# 평균 확률 계산
mean_prob = np.mean(all_probs, axis=0)

# 등급 매핑
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

# 성능 평가
grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test
})

summary = grade_df.groupby("grade", dropna=False).agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

# 결과 확인
print(summary)


  grade     n  default  expected_prob       odr  abs_diff      wmae
0   AAA     0        0            NaN       NaN       NaN       NaN
1    AA   193        0       0.042670  0.000000  0.042670  0.000888
2     A   449        0       0.072033  0.000000  0.072033  0.003488
3   BBB   996       14       0.175490  0.014056  0.161433  0.017339
4     B  7635      208       0.503524  0.027243  0.476281  0.392150


<ipython-input-40-f77da2d101c3>:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade", dropna=False).agg(


In [4]:
# 100회 반복 수행
#xg boost->logistic regression
all_probs = []

for repeat in range(100):
    X_train, y_train, X_test, y_test = get_basic_split(df, wl=5, seed=repeat)

    xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=repeat)
    xgb.fit(X_train, y_train)
    train_prob = xgb.predict_proba(X_train)[:, 1].reshape(-1, 1)
    test_prob = xgb.predict_proba(X_test)[:, 1].reshape(-1, 1)

    lr = LogisticRegression(max_iter=1000, random_state=repeat)
    lr.fit(train_prob, y_train)
    final_prob = lr.predict_proba(test_prob)[:, 1]
    all_probs.append(final_prob)

# 평균 확률 계산 및 등급 평가
mean_prob = np.mean(all_probs, axis=0)
bins = [0, 0.02, 0.05, 0.1, 0.25, 1.0]
labels = ["AAA", "AA", "A", "BBB", "B"]
grades = pd.cut(mean_prob, bins=bins, labels=labels)

grade_df = pd.DataFrame({
    "prob": mean_prob,
    "grade": grades,
    "actual": y_test
})

summary = grade_df.groupby("grade", dropna=False).agg(
    n=("actual", "count"),
    default=("actual", "sum"),
    expected_prob=("prob", "mean")
).reset_index()

summary["odr"] = summary["default"] / summary["n"]
summary["abs_diff"] = np.abs(summary["expected_prob"] - summary["odr"])
summary["wmae"] = (summary["abs_diff"] * summary["n"]) / summary["n"].sum()

# 결과 확인
print(summary)



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:50:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

  grade     n  default  expected_prob       odr  abs_diff      wmae
0   AAA  1601        4       0.009197  0.002498  0.006698  0.001156
1    AA   835        2       0.033512  0.002395  0.031117  0.002802
2     A   849        6       0.072802  0.007067  0.065735  0.006018
3   BBB  1474       14       0.166344  0.009498  0.156846  0.024932
4     B  4514      196       0.625130  0.043420  0.581710  0.283170


<ipython-input-4-3f2df9d76ab0>:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = grade_df.groupby("grade", dropna=False).agg(
